In [12]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
import os
import xgboost as xgb
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

In [10]:
df = pd.read_csv("./dataset/Test_dataset.csv")
df.drop(columns=['DTS', 'SGR', 'ROPA', 'RMIC'],inplace=True)
df

,uuid,file_name,BS,PEF,RXO,DCAL,ROP,SP,RMED,GR,...,DRHO,DEPTH_MD,RHOB,NPHI,X_LOC,CALI,RSHA,Z_LOC,RDEP,DEPT
0,14812,25_10-10,12.250001,3.125396,NaN,0.051304,45.224689,NaN,0.992759,61.177399,...,0.010112,1348.310425,2.131319,0.398266,459853.34375,12.301304,0.985707,-1308.193359,1.019758,1348.310400
1,72908,25_10-10,12.250001,3.292384,NaN,0.049227,49.948738,NaN,0.896075,63.512333,...,0.009631,1348.462402,2.129777,0.389460,459853.34375,12.299226,0.888855,-1308.345337,0.924186,1348.462400
2,103104,25_10-10,12.250001,3.693287,NaN,0.047644,52.813866,NaN,0.877702,63.515835,...,0.008258,1348.614380,2.138082,0.394868,459853.34375,12.297644,0.870774,-1308.497314,0.895908,1348.614400
3,23480,25_10-10,12.250001,4.051715,NaN,0.026056,58.635353,NaN,0.928396,63.153057,...,0.015513,1348.766357,2.153999,0.389355,459853.34375,12.276056,0.913030,-1308.649292,0.955895,1348.766400
4,104267,25_10-10,12.250001,4.299806,NaN,0.049645,58.242565,NaN,0.990412,60.224148,...,0.016630,1348.918457,2.140920,0.365808,459853.34375,12.299644,0.959649,-1308.801392,1.022451,1348.918400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121792,10928,34_3-3 A,8.500000,8.879548,NaN,NaN,15.756256,NaN,NaN,65.925987,...,0.043345,NaN,2.515146,NaN,NaN,8.647468,NaN,NaN,NaN,5006.809976
121793,26727,34_3-3 A,8.500000,10.237462,NaN,NaN,15.760216,NaN,NaN,66.297127,...,0.053834,NaN,2.543073,NaN,NaN,8.636636,NaN,NaN,NaN,5006.961976
121794,83995,34_3-3 A,8.500000,10.901587,NaN,NaN,15.990236,NaN,NaN,64.853714,...,0.055133,NaN,2.576337,NaN,NaN,8.592650,NaN,NaN,NaN,5007.113976
121795,67375,34_3-3 A,8.500000,10.468700,NaN,NaN,15.314774,NaN,NaN,62.779541,...,0.047498,NaN,2.606787,NaN,NaN,8.546233,NaN,NaN,NaN,5007.265976


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121797 entries, 0 to 121796
Data columns (total 23 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   uuid       121797 non-null  int64  
 1   file_name  121797 non-null  object 
 2   BS         89910 non-null   float64
 3   PEF        86459 non-null   float64
 4   RXO        6707 non-null    float64
 5   DCAL       41164 non-null   float64
 6   ROP        87913 non-null   float64
 7   SP         75533 non-null   float64
 8   RMED       107922 non-null  float64
 9   GR         121794 non-null  float64
 10  DTC        120938 non-null  float64
 11  MUDWEIGHT  33364 non-null   float64
 12  Y_LOC      114368 non-null  float64
 13  DRHO       95297 non-null   float64
 14  DEPTH_MD   114368 non-null  float64
 15  RHOB       96178 non-null   float64
 16  NPHI       78221 non-null   float64
 17  X_LOC      114368 non-null  float64
 18  CALI       121790 non-null  float64
 19  RSHA       62370 non-nu

In [14]:
class PreprocessingData:
    def __init__(self, df):
        """
        Initialize the PreprocessingData class with a DataFrame.
        """
        self.df = df.copy()

    def interpolate_data(self):
        """
        Perform interpolation and mean imputation on depth, spatial, and log features.
        """
        depth_features = ['DEPTH_MD', 'DEPT']
        for col in depth_features:
            if col in self.df.columns:
                self.df[col] = self.df[col].interpolate()

        spatial_features = ['X_LOC', 'Y_LOC', 'Z_LOC']
        for col in spatial_features:
            if col in self.df.columns:
                self.df[col].fillna(self.df[col].mean(), inplace=True)

        log_features = ['GR', 'SP', 'CALI']
        for col in log_features:
            if col in self.df.columns:
                self.df[col] = self.df[col].interpolate()

    def predict_missing_sonic_logs(self):
        """
        Predict missing values in sonic logs (DTC) using linear regression.
        """
        for col in self.df.columns:
            self.df[col] = pd.to_numeric(self.df[col], errors='coerce')

        target = 'DTC'
        if target in self.df.columns:
            temp_df = self.df.dropna(subset=['RHOB', 'NPHI', target])
            if not temp_df.empty:
                model = LinearRegression().fit(temp_df[['RHOB', 'NPHI']], temp_df[target])
                missing_idx = self.df[target].isna() & self.df['RHOB'].notna() & self.df['NPHI'].notna()
                if missing_idx.sum() > 0:
                    self.df.loc[missing_idx, target] = model.predict(self.df.loc[missing_idx, ['RHOB', 'NPHI']])
            self.df[target].fillna(self.df[target].median(), inplace=True)

    def apply_feature_engineering(self):
        """
        Apply various feature engineering techniques:
        - Log transformation
        - Crossplot differences
        - Resistivity ratios
        - Rolling statistics
        - First and second derivatives
        - Euclidean distance computation
        """
        resistivity_features = ['RSHA', 'RDEP', 'RMED']
        for col in resistivity_features:
            if col in self.df.columns:
                self.df[f'log_{col}'] = np.log1p(self.df[col])
                self.df[col].fillna(self.df[col].median(), inplace=True)

        if {'NPHI', 'RHOB'}.issubset(self.df.columns):
            self.df['DNC'] = self.df['NPHI'] - self.df['RHOB']

        if {'RHOB', 'DTC'}.issubset(self.df.columns):
            self.df['AI'] = self.df['RHOB'] / self.df['DTC']

        if {'RDEP', 'RSHA'}.issubset(self.df.columns):
            self.df['R_ratio'] = self.df['RDEP'] / self.df['RSHA']
            self.df['R_contrast'] = self.df['RDEP'] - self.df['RSHA']

        if 'GR' in self.df.columns:
            self.df['GR_rolling_mean'] = self.df['GR'].rolling(window=5, min_periods=1).mean()
        if 'RHOB' in self.df.columns:
            self.df['RHOB_rolling_std'] = self.df['RHOB'].rolling(window=5, min_periods=1).std()

        if 'GR' in self.df.columns:
            self.df['GR_grad'] = np.gradient(self.df['GR'])
            self.df['GR_grad2'] = np.gradient(self.df['GR_grad'])
        if 'RHOB' in self.df.columns:
            self.df['RHOB_grad'] = np.gradient(self.df['RHOB'])
            self.df['RHOB_grad2'] = np.gradient(self.df['RHOB_grad'])

        if {'X_LOC', 'Y_LOC', 'Z_LOC'}.issubset(self.df.columns):
            x_mean, y_mean, z_mean = self.df['X_LOC'].mean(), self.df['Y_LOC'].mean(), self.df['Z_LOC'].mean()
            self.df['distance_to_mean'] = np.sqrt((self.df['X_LOC'] - x_mean) ** 2 +
                                                  (self.df['Y_LOC'] - y_mean) ** 2 +
                                                  (self.df['Z_LOC'] - z_mean) ** 2)

    def preprocess(self):
        """
        Run all preprocessing steps in order.
        """
        self.interpolate_data()
        self.predict_missing_sonic_logs()
        self.apply_feature_engineering()
        print("Feature engineering and preprocessing completed successfully!")
        return self.df






In [15]:
class FeatureEngineering:
    def __init__(self, df):
        """
        Initialize the FeatureEngineering class with a DataFrame.
        """
        self.df = df.copy()

    def compute_vclay(self):
        """Compute Vclay from Gamma Ray (GR) log."""
        if 'GR' in self.df.columns:
            GR_min, GR_max = self.df['GR'].min(), self.df['GR'].max()
            self.df['Vclay'] = (self.df['GR'] - GR_min) / (GR_max - GR_min)

    def compute_density_porosity(self):
        """Compute Density Porosity (PHI_D) from bulk density (RHOB)."""
        rho_matrix, rho_fluid = 2.65, 1.0  # Assumed matrix and fluid density values
        if 'RHOB' in self.df.columns:
            self.df['PHI_D'] = (rho_matrix - self.df['RHOB']) / (rho_matrix - rho_fluid)

    def compute_sonic_porosity(self):
        """Compute Sonic Porosity (PHI_S) from sonic transit time (DTC)."""
        dt_matrix, dt_fluid = 55.5, 189.0  # Assumed matrix and fluid travel times
        if 'DTC' in self.df.columns:
            self.df['PHI_S'] = (self.df['DTC'] - dt_matrix) / (dt_fluid - dt_matrix)

    def compute_ndpd(self):
        """Compute Neutron-Density Porosity Difference (NDPD)."""
        if 'NPHI' in self.df.columns and 'PHI_D' in self.df.columns:
            self.df['NDPD'] = self.df['NPHI'] - self.df['PHI_D']

    def compute_water_saturation(self):
        """Compute Water Saturation (Sw) using Archie's Equation."""
        if all(col in self.df.columns for col in ['RDEP', 'PHI_D']):
            a, m, n, rw = 1.0, 2.0, 2.0, 0.1  # Archie's parameters
            self.df['Sw'] = (a * rw / self.df['RDEP']) ** (1/n) / self.df['PHI_D'] ** (m/n)

    def apply_feature_engineering(self):
        """Run all feature engineering functions."""
        self.compute_vclay()
        self.compute_density_porosity()
        self.compute_sonic_porosity()
        self.compute_ndpd()
        self.compute_water_saturation()
        print("Feature engineering completed successfully!")
        return self.df



In [16]:
# Initialize PreprocessingData class and apply preprocessing
preprocessor = PreprocessingData(df)
df_processed = preprocessor.preprocess()

# Initialize FeatureEngineering class and apply feature engineering
feature_engineer = FeatureEngineering(df_processed)
df_engineered = feature_engineer.apply_feature_engineering()

df_engineered

Feature engineering and preprocessing completed successfully!
Feature engineering completed successfully!


,uuid,file_name,BS,PEF,RXO,DCAL,ROP,SP,RMED,GR,...,GR_grad,GR_grad2,RHOB_grad,RHOB_grad2,distance_to_mean,Vclay,PHI_D,PHI_S,NDPD,Sw
0,14812,NaN,12.250001,3.125396,NaN,0.051304,45.224689,NaN,0.992759,61.177399,...,2.334934,-1.165716,-0.001542,0.004923,8.194856e+04,0.056710,0.314352,0.584547,0.083914,0.996173
1,72908,NaN,12.250001,3.292384,NaN,0.049227,49.948738,NaN,0.896075,63.512333,...,1.169218,-1.257286,0.003381,0.006827,8.194856e+04,0.058878,0.315287,0.584461,0.074174,1.043313
2,103104,NaN,12.250001,3.693287,NaN,0.047644,52.813866,NaN,0.877702,63.515835,...,-0.179638,-1.407531,0.012111,-0.000981,8.194856e+04,0.058881,0.310254,0.563593,0.084614,1.076841
3,23480,NaN,12.250001,4.051715,NaN,0.026056,58.635353,NaN,0.928396,63.153057,...,-1.645844,-1.960062,0.001419,-0.012699,8.194855e+04,0.058544,0.300606,0.543627,0.088749,1.075961
4,104267,NaN,12.250001,4.299806,NaN,0.049645,58.242565,NaN,0.990412,60.224148,...,-4.099762,-1.721710,-0.013287,-0.004078,8.194855e+04,0.055824,0.308534,0.494044,0.057275,1.013623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121792,10928,NaN,8.500000,8.879548,NaN,NaN,15.756256,92.698509,1.431406,65.925987,...,-0.148232,-0.357088,0.019808,0.011565,5.820944e-11,0.061119,0.081729,0.321650,NaN,3.145783
121793,26727,NaN,8.500000,10.237462,NaN,NaN,15.760216,92.698509,1.431406,66.297127,...,-0.536137,-0.805281,0.030595,0.006024,5.820944e-11,0.061464,0.064804,0.321650,NaN,3.967394
121794,83995,NaN,8.500000,10.901587,NaN,NaN,15.990236,92.698509,1.431406,64.853714,...,-1.758793,-0.956762,0.031857,-0.001645,5.820944e-11,0.060124,0.044644,0.321650,NaN,5.758899
121795,67375,NaN,8.500000,10.468700,NaN,NaN,15.314774,92.698509,1.431406,62.779541,...,-2.449661,-0.533178,0.027306,-0.003848,5.820944e-11,0.058198,0.026189,0.321650,NaN,9.817052


In [30]:
import joblib
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the trained XGBoost model
model = joblib.load("xgboost_model.pkl")

# Load the trained LabelEncoder (this assumes you saved it as "label_encoder.pkl")
le = joblib.load("label_encoder.pkl")

df_test = df_engineered[['NDPD', 'R_ratio', 'Y_LOC', 'X_LOC', 'PEF', 'BS', 'RXO', 'distance_to_mean',
               'Sw', 'RSHA', 'MUDWEIGHT', 'RMED', 'CALI', 'SP', 'GR_rolling_mean', 'DCAL', 'GR_grad2', 'RHOB_rolling_std',
               'DRHO', 'ROP', 'RHOB_grad', 'RHOB_grad2', 'GR_grad']]

# Ensure the features are aligned with training
X_test = df_test  # Adjust this based on selected features
predictions = model.predict(X_test)

# Convert predictions into a DataFrame
df_predictions = pd.DataFrame({'Predictions': predictions})

# Convert numerical predictions back to original labels
predicted_labels = le.inverse_transform(predictions.astype(int))  # Ensure dtype is int

# Convert predictions into a DataFrame
df_predictions = pd.DataFrame({'uuid': df_engineered['uuid'], 'Predictions': predicted_labels})

# Save predictions to CSV
#df_predictions.to_csv("xgboost_predictions.csv", index=False)

#print("Predictions saved successfully as 'xgboost_predictions.csv'!")

df_predictions


c:\Users\user.pupuk\AppData\Local\anaconda3\envs\hedging\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,uuid,Predictions
0,14812,65000.0
1,72908,65030.0
2,103104,65000.0
3,23480,30000.0
4,104267,70000.0
...,...,...
121792,10928,70000.0
121793,26727,70000.0
121794,83995,65030.0
121795,67375,65030.0


In [31]:
df_label = pd.read_csv("./dataset/Labels_code.csv")
df_label

,label_code,label_name
0,30000,Sandstone
1,65030,Sandstone/Shale
2,65000,Shale
3,80000,Marl
4,74000,Dolomite
5,70000,Limestone
6,70032,Chalk
7,88000,Halite
8,86000,Anhydrite
9,99000,Tuff


In [32]:
# Replace numeric predictions with label names using df_label
df_predictions = df_predictions.merge(df_label, left_on="Predictions", right_on="label_code", how="left")

# Drop the numeric predictions column and rename "label_name"
df_predictions = df_predictions.drop(columns=["Predictions", "label_code"])




In [33]:
df_predictions

,uuid,label_name
0,14812,Shale
1,72908,Sandstone/Shale
2,103104,Shale
3,23480,Sandstone
4,104267,Limestone
...,...,...
121792,10928,Limestone
121793,26727,Limestone
121794,83995,Sandstone/Shale
121795,67375,Sandstone/Shale


In [34]:
# Save to CSV
df_predictions.to_csv("submission.csv", index=False)

print("Predictions saved successfully as 'xgboost_predictions.csv'!")

Predictions saved successfully as 'xgboost_predictions.csv'!
